In [ ]:
!python ~/MasterProject/Code/ClinicaTools/AD-DL/clinicaaddl/clinicaaddl/main.py train /u/horlavanasta/MasterProject//DataAndExperiments/Experiments/Experiments-1.5T-3T/NNs_Bayesian/ResNet18/subject_model-ResNet18_preprocessing-linear_task-AD_CN_norm-1_loss-WeightedCrossEntropy_augmTrue --n_splits 1 --split 0  --batch_size 5               

In [ ]:
def show_fpg(data_batch, indices=None,plane="sag", num_rows=2, 
    num_cols=2, name=None, folder="/current_augmentations_examples/"):
    
    import matplotlib.pyplot as plt
    import numpy as np
    
    fig, axes = plt.subplots(num_rows, num_cols, figsize=((int(8 * num_rows), int(6 * num_cols))))
    print(data_batch.shape)
    data_batch=data_batch[:num_rows*num_cols].reshape(num_rows, num_cols, data_batch.shape[1],  data_batch.shape[2],  data_batch.shape[3], 
                                  data_batch.shape[4])
    print(data_batch.shape)

    for row in range(num_rows):
        for col in range(num_cols):

            i, j, k = indices
            data=data_batch[row][col]
            kwargs = dict(cmap='gray', interpolation='none')
            slices=dict()
            slices["sag"], slices["cor"], slices["axi"] = np.rot90(data[0, i]), np.rot90(data[0, :, j]), np.rot90(data[0, ..., k])

            axes[row][col].imshow(slices[plane],**kwargs)
            axes[row][col].axis('off')

    if name is not None:
        fig.suptitle(name)
    plt.subplots_adjust( left=0.05, right=0.95, top=0.95, bottom=0.05, wspace=0.05, hspace=0.05)
            
#     path = '../../outputs/'+folder
#     if not os.path.exists(path):
#         os.makedirs(path)
#     plt.savefig(path + str(name) + '.png')
    plt.show()
    plt.close()

In [54]:
def translate_parameters(args,batch_size=8 ):
    """
    Translate the names of the parameters between command line and source code.
    """
    args.gpu = False
    args.num_workers = args.nproc
    args.optimizer = "Adam"
    args.batch_size=batch_size
    # args.loss = "default"

    if hasattr(args, "caps_dir"):
        args.input_dir = args.caps_dir
    if hasattr(args, "unnormalize"):
        args.minmaxnormalization = not args.unnormalize
    if hasattr(args, "slice_direction"):
        args.mri_plane = args.slice_direction
    if hasattr(args, "network_type"):
        args.mode_task = args.network_type

    if not hasattr(args, "selection_threshold"):
        args.selection_threshold = None
        
    if not hasattr(args, "verbose"):
        args.verbose = 0
    if not hasattr(args, "bayesian"):
        args.bayesian = False

    if not hasattr(args, "prepare_dl"):
        if hasattr(args, "use_extracted_features"):
            args.prepare_dl = args.use_extracted_features
        elif hasattr(args, "use_extracted_patches") and args.mode == "patch":
            args.prepare_dl = args.use_extracted_patches
        elif hasattr(args, "use_extracted_slices") and args.mode == "slice":
            args.prepare_dl = args.use_extracted_slices
        elif hasattr(args, "use_extracted_roi") and args.mode == "roi":
            args.prepare_dl = args.use_extracted_roi

    return args

def show_fpg_ax(img, axes, indices=None, plane="sag"):
    
    import matplotlib.pyplot as plt
    import numpy as np
    print(img.shape)
    if indices is None: 
        i, j, k = img.shape[0]//2, img.shape[1]//2, img.shape[2]//2
    
    kwargs = dict(cmap='gray', interpolation='none')
    slices=dict()
    slices["sag"], slices["cor"], slices["axi"] = np.rot90(img[i]), np.rot90(img[:, j]), np.rot90(img[..., k])
    axes.imshow(slices[plane],cmap='gray', interpolation='none')
    axes.axis('off')

    
def get_unprocessed_images(participant_id, session_id, bids_directory="/u/horlavanasta/MasterProject/DataAndExperiments/Data/BIDS"):
    import numpy as np
    import nibabel as nib
    imgs_before=[]
    for i in range(len(participant_id)):
        image_path=os.path.join(bids_directory,  participant_id[i], session_id[i], "anat",participant_id[i] + '_' + session_id[i]
                                   +'_T1w.nii.gz')
        image_nii = nib.load(image_path)
        image_np = image_nii.get_fdata()
        imgs_before.append(image_np)
    return imgs_before
        
    
def show_preprocessing(imgs_before, imgs_after, num_cols=4, plane="sag", MS="1.5T"):
    import matplotlib.pyplot as plt
    import numpy as np
    fig, axes = plt.subplots(2, num_cols, 
                             figsize=((int(8 *num_cols), int(8 * 2)))
                            )
    for i in range(num_cols):
        show_fpg_ax(imgs_before[i],axes[0][i] )
        show_fpg_ax(imgs_after[i][0],axes[1][i])
#     plt.subplots_adjust( left=0.01, right=0.99, top=0.99, bottom=0.01)

    plt.subplots_adjust(left=0.01, right=0.99, top=0.99, bottom=0.01, wspace=0.02, hspace=0.02)
    path = '../../plots'
    plt.savefig(os.path.join(path, 'preprocessing_%s.png'%MS))
    plt.close()
    
def show_data(model_folder, name=None, plane="sag", show_augmentations=False, MS="1.5T"):
    from tools.deep_learning.models import init_model
    from tools.deep_learning.data import (get_transforms,
                                        load_data,
                                        return_dataset,
                                        generate_sampler)
    from tools.deep_learning.iotools import return_logger
    from argparse import Namespace
    from torch.utils.data import DataLoader
    import torch


    path_params = os.path.join(model_folder, "commandline_train.json")
    with open(path_params, "r") as f:
        params = json.load(f)
    params = translate_parameters(Namespace(**params))
    main_logger = return_logger(params.verbose, "main process")

    
    train_transforms, all_transforms = get_transforms(params.mode,
                                                      minmaxnormalization=params.minmaxnormalization,
                                                      data_augmentation=None,
                                                      output_dir=None)
    training_df, valid_df = load_data(
            params.tsv_path,
            params.diagnoses,
            0,
            n_splits=params.n_splits,
            baseline=params.baseline,
            logger=main_logger
        )

    
    data_valid = return_dataset(params.mode, params.input_dir, valid_df, params.preprocessing,
                                train_transformations=train_transforms, all_transformations=all_transforms,
                                params=params)

    
    valid_loader = DataLoader(
        data_valid,
        batch_size=params.batch_size,
        shuffle=False,
        num_workers=params.num_workers,
        pin_memory=True
    )
    
    sample = next(iter(valid_loader))
    sample = next(iter(valid_loader))
#     show_fpg(sample["image"].numpy(), name=name, plane=plane)
    participant_id, session_id =sample['participant_id'],sample["session_id"]
    imgs_after=sample["image"].numpy()
    imgs_before=get_unprocessed_images(participant_id, session_id)
#     print(imgs_after[0].shape)
    show_preprocessing(imgs_before, imgs_after, num_cols=4, plane="sag", MS=MS)
    
    

In [55]:
import pathlib
import pandas as pd
import os
import json

folders = []
MS_main_list = ["3T",]
MS_list_dict = {'1.5T':['1.5T', '3T'], "3T": ['3T', '1.5T'], "1.5T-3T": ["1.5T-3T"]}
home_folder='/u/horlavanasta/MasterProject/'

isBayesian=True
for MS in MS_main_list[:]:
    print("MS %s \n ____________________________________________________________________________________________"%MS)
    model_types = ["ResNet18"]
    
    model_dir_general = os.path.join(home_folder,"DataAndExperiments/Experiments_5-fold/Experiments-" + MS, "NNs" if isBayesian else "NNs")
    for network in model_types[:]:
        model_dir = os.path.join(model_dir_general, network)
        # output_dir = pathlib.Path(output_dir)
        modelPatter = "subject_model*"
        folders = [f for f in pathlib.Path(model_dir).glob(modelPatter)]

        for f in folders[:1]:
            
            print(f)
#             show_model(f)
            show_data(f, name=None, plane="sag", show_augmentations=False, MS=MS)
#             show_data(f, plane="sag")
#             show_data(f, plane="cor")
#             show_data(f, plane="axi")
             

MS 3T 
 ____________________________________________________________________________________________
/u/horlavanasta/MasterProject/DataAndExperiments/Experiments_5-fold/Experiments-3T/NNs/ResNet18/subject_model-ResNet18_preprocessing-linear_task-AD_CN_norm-1_loss-WeightedCrossEntropy_augmFalse_20210612_195440
(196, 256, 256)
(169, 208, 179)
(176, 240, 256)
(169, 208, 179)
(196, 256, 256)
(169, 208, 179)
(196, 256, 256)
(169, 208, 179)


In [ ]:
hl_graph